In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import CSVLogger
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('../datas/RGB_color_labels.csv') # read the dataset using pandas library
dataset = pd.get_dummies(dataset, columns=['label']) # adds 11 new columns that one hot encodes the class of the color

In [3]:
train_dataset = dataset.sample(frac=0.8, random_state=8) #train_dataset = 80% of total dataset  
#random_state = any int value means every time when you run your program you will get the same output for train and test dataset, random_state is None by default which means every time when you run your program you will get different output because of splitting between train and test varies within 
test_dataset = dataset.drop(train_dataset.index) #remove train_dataset from dataframe to get test_dataset


# Separates the dataset(that will be the input) and the labels(that will be the class of the input values). This part is for training.
train_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    train_labels[x] = train_dataset.pop(x)


# Separates the dataset(that will be the input) and the labels(that will be the class of the input values). This part is for testing.
test_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    test_labels[x] = test_dataset.pop(x)


# Converts all data into numpy arrays
# Why convert? Because the code gives a lot of errors if you did not input a numpy or tensors array to the neural network.
# I will leave to the reader to research about tensors in the documentation. https://www.tensorflow.org/guide/tensor
train_dataset = np.array(train_dataset)
train_labels = np.array(train_labels)
train_labels = np.argmax(train_labels, axis=1)

test_dataset = np.array(test_dataset)
test_labels = np.array(test_labels)
test_labels = np.argmax(test_labels, axis=1)

In [4]:
class ColorClassifierModel:
    def __init__(self, model_path=None) -> None:
        self.model_path = model_path
        self.train_path = f"{self.model_path}/saved_per_train" # The train path of the model
        self.least_loss_checkpoint_path = self.return_path() # The file path of the least loss checkpoint
        self.loaded_model = tf.keras.models.load_model(self.least_loss_checkpoint_path) # Loads the least loss checkpoint in a variable

    # This function can be called if you want to create a fresh new model of color classifier
    @staticmethod
    def make_model(train_dataset, train_labels, validation_data=None, num_of_inputs=3, num_of_outputs=11, num_of_hidden_layers=5, num_of_neurons_of_hidden_layers=15, saved_models_path="../trained_models"):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(num_of_inputs, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=(num_of_inputs,)))
        for i in range(num_of_hidden_layers):
            model.add(tf.keras.layers.Dense(num_of_neurons_of_hidden_layers, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
        model.add(tf.keras.layers.Dense(num_of_outputs))
        model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
        model.build()

        from datetime import datetime
        if os.listdir(f"{saved_models_path}"):
            # Makes the model folder
            last_model_num = os.listdir(f"{saved_models_path}")[-1][-1]
            os.mkdir(f"{saved_models_path}/model{int(last_model_num) + 1}")   
            # Makes a readme file that contains information on when was this created
            last_model = os.listdir(f"{saved_models_path}")[-1]
            with open(f"{saved_models_path}/{last_model}/readme.txt", "w+") as f:
                f.write(f"Model created on: {str(datetime.now())}")

        # If there are no models on path
        elif not os.listdir(f"{saved_models_path}"):
            # Makes the model folder
            os.mkdir(f"{saved_models_path}/model1")
            # Makes a readme file that contains information on when was this created
            with open(f"{saved_models_path}/model1/readme.txt", "w+") as f:
                f.write(f"Model created on: {str(datetime.now())}")

        # Trains the newly created model
        from tensorflow.keras.callbacks import CSVLogger
        os.makedirs(f"{saved_models_path}/{last_model}/saved_per_train/train1")
        checkpoint_path = f"{saved_models_path}/{last_model}/saved_per_train/train1" + "/Epoch{epoch:02d}_loss{loss:.2f}"
        csv_logger = CSVLogger(f"{saved_models_path}/{last_model}/saved_per_train/train1/logs.csv", separator=',', append=False)

        # Create a callback that saves the model's weights
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                        verbose=1,
                                                        monitor='accuracy',
                                                        save_freq=100) # if save_freq='epochs' it saves the model per epoch
                                                                        # if save_freq=int_type it saves the model per <int_type> of batches
        # Train the model with the new callback
        model.fit(train_dataset, 
                train_labels,  
                epochs=100,
                batch_size = 1000,
                validation_data=validation_data,
                callbacks=[cp_callback, csv_logger], # Pass callback to training
                shuffle=True)  

        # This may generate warnings related to saving the state of the optimizer.
        # These warnings (and similar warnings throughout this notebook)
        # are in place to discourage outdated usage, and can be ignored.


    def train_model(self, train_dataset, train_labels, epochs=100, batch_size=1000, save_freq=100, save_param_excel=False, custom_checkpoint=None):
        if not custom_checkpoint: # Checks if user specified a custom checkpoint path to begin
            self.least_loss_checkpoint_path = self.return_path() # The file path of the least loss tensor model
            self.loaded_model = tf.keras.models.load_model(self.least_loss_checkpoint_path) # Loads the least loss tensor model in a variable
        elif custom_checkpoint:
            self.loaded_model = tf.keras.models.load_model(custom_checkpoint)

        # This part makes a new folder for the new train batch
        self.last_model_num = os.listdir(f"{self.train_path}")[-1][-1] # Gets the number of the latest train folder
        os.makedirs(f"{self.train_path}/train{int(self.last_model_num) + 1}") # Makes a new folder for the current train batch
        self.last_train_folder = f"{self.train_path}/train{int(self.last_model_num) + 1}" # The relative path of the current train batch folder
 
        # Create a callback that saves the model's weights
        from tensorflow.keras.callbacks import CSVLogger
        checkpoint_path = f"{self.last_train_folder}" + "/Epoch{epoch:02d}_loss{loss:.2f}" # Makes a folder and saves the model for every batch of epochs. Save frequency depends on save_freq parameter
        csv_logger = CSVLogger(f"{self.last_train_folder}/logs.csv", separator=',', append=False)

        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 verbose=1,
                                                 monitor='accuracy',
                                                 save_freq=save_freq) # if save_freq='epochs' it saves the model per epoch
                                                                      # if save_freq=int_type it saves the model per <int_type> of batches

        self.loaded_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
        # Train the model with the new callback
        self.loaded_model.fit(train_dataset, 
                train_labels,  
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(test_dataset, test_labels),
                callbacks=[cp_callback, csv_logger], # Pass callback to training
                shuffle=True) 
        # This may generate warnings related to saving the state of the optimizer.
        # These warnings (and similar warnings throughout this notebook)
        # are in place to discourage outdated usage, and can be ignored.    
        if save_param_excel:
            self.make_excel()


    def make_excel(self, specific_train=None):
        # This function saves the weights and biases each of the model of the train in a excel file
        import xlsxwriter
        if not specific_train:
            self.last_model_num = os.listdir(f"{self.train_path}")[-1][-1]
            self.train_folder = f"{self.train_path}/train{int(self.last_model_num)}"
            last_trained_checkpoint_list = os.listdir(self.last_train_folder)
            try:
                last_trained_checkpoint_list.remove('logs.csv')
            except:
                pass
        elif specific_train:
            self.train_folder = f"{self.train_path}/{specific_train}"
            last_trained_checkpoint_list = os.listdir(f"{self.train_path}/{specific_train}")
            try:
                last_trained_checkpoint_list.remove('logs.csv')
            except:
                pass
            
        for model in last_trained_checkpoint_list:
            loaded_model = tf.keras.models.load_model(f"{self.train_folder}/{model}")

            relative_row_idx = 0 # this is for writing bias
            row_idx = 0
            max_row_idx = 0
            column_idx = 0

            workbook = xlsxwriter.Workbook(f"{self.train_folder}/{model}/saved_weights_biases.xlsx")
            worksheet = workbook.add_worksheet()

            for layer in loaded_model.layers:
                for row_weights in layer.get_weights()[0].T: # the reason I transposed the matrix because tensorflow makes weights in transposed position of matrix
                    for weights in row_weights:
                        worksheet.write(row_idx, column_idx, weights)
                        column_idx += 1

                    column_idx += 1
                    worksheet.write(row_idx, column_idx, layer.get_weights()[1].T[relative_row_idx]) # the reason I transposed the matrix because tensorflow makes weights in transposed position of matrix
                    relative_row_idx += 1
                    row_idx += 1
                    column_idx = 0

                relative_row_idx = 0
                row_idx += 2
            workbook.close()

    def return_path(self, custom_train=None):
        '''
        Defaults to returning the latest train with the least loss model relative file path
        '''
        last_trained = os.listdir(self.train_path)[-1] # This gives the latest trained
        checkpoint_list = os.listdir(f"{self.train_path}/{last_trained}") # List all the saved trains
        try:
            checkpoint_list.remove("logs.csv") # Removes the logs.csv file on the list because we do not need it we only need the model folders
        except:
            print("logs.csv does not exist")
        least_loss_model = min(checkpoint_list, key=lambda loss_val:loss_val[-4:-1]) # Finds the model which has the least loss in the list
        least_loss_checkpoint_path = f"{self.train_path}/{last_trained}/{least_loss_model}" # The file path of the least loss tensor model
        return least_loss_checkpoint_path


In [6]:
load_model4 = ColorClassifierModel("../trained_models/model4")

In [7]:
load_model4.train_model(train_dataset, train_labels, epochs=500, save_freq=500, save_param_excel=True)

Epoch 1/500
5/5 [==============================] - 2s 187ms/step - loss: 0.8595 - accuracy: 0.7457 - val_loss: 0.8169 - val_accuracy: 0.7465
Epoch 2/500
5/5 [==============================] - 0s 32ms/step - loss: 0.7364 - accuracy: 0.7714 - val_loss: 0.7815 - val_accuracy: 0.7594
Epoch 3/500
5/5 [==============================] - 0s 24ms/step - loss: 0.7442 - accuracy: 0.7808 - val_loss: 0.7629 - val_accuracy: 0.7802
Epoch 4/500
5/5 [==============================] - 0s 23ms/step - loss: 0.7620 - accuracy: 0.7595 - val_loss: 0.7103 - val_accuracy: 0.7861
Epoch 5/500
5/5 [==============================] - 0s 25ms/step - loss: 0.7242 - accuracy: 0.7803 - val_loss: 0.7145 - val_accuracy: 0.7891
Epoch 6/500
5/5 [==============================] - 0s 23ms/step - loss: 0.6919 - accuracy: 0.7971 - val_loss: 0.7041 - val_accuracy: 0.8010
Epoch 7/500
5/5 [==============================] - 0s 28ms/step - loss: 0.6850 - accuracy: 0.8083 - val_loss: 0.6839 - val_accuracy: 0.8089
Epoch 8/500
5/5 [==